# Chat Models大语言模型

## 简介

在Agent智能体中，大语言模型提供理解、决策、调度等能力，是最基础的大脑模块。因此，本文我们介绍如何上手使用大语言模型。

简而言之，大语言模型基于深度学习技术，能够自动理解和生成人类自然语言文本的模型，广泛应用于文本生成、机器翻译、自动摘要、问答系统、情感分析等任务。

目前市面上已经有很多大语言模型，比如OpenAI的ChatGPT、百度的文心一言等等。大语言模型通常具有大量参数，对运行机器的显存和算量有很高的要求。为了方便大家使用，这些大语言模型都有对外提供相应的调用接口。

在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。

| 模型名称 | 说明 | 功能 | 输入token数量上限 |
|:--- | :--- | :--- | :--- |
| ernie-3.5 | 文心大模型3.5版本。具备优秀的知识增强和内容生成能力，在文本创作、问答、推理和代码生成等方面表现出色。 | 对话补全，函数调用 | 3000 |
| ernie-turbo | 文心大模型。相比ernie-3.5模型具备更快的响应速度和学习能力，API调用成本更低。 | 对话补全 |  3000 |
| ernie-4.0 | 文心大模型4.0版本，具备目前系列模型中最优的理解和生成能力。 | 对话补全，函数调用 |  3000 |
| ernie-longtext | 文心大模型。在ernie-3.5模型的基础上增强了对长对话上下文的支持，输入token数量上限为7000。 | 对话补全，函数调用 |  7000 |

## 上手使用文心一言

### 安装

大家可以参考[官方文档](https://github.com/PaddlePaddle/ERNIE-Bot-SDK)，进行安装ERNIE Bot SDK。



### 鉴权

大家在使用ERNIE Bot Agent之前，需要完成鉴权步骤：

* 在[AI Studio星河社区](https://aistudio.baidu.com/index)注册并登录账号
* 在个人中心的[访问令牌页面](https://aistudio.baidu.com/index/accessToken)获取用户凭证`Access Token`
* 通过环境变量或者`Python`代码设置`Access Token`

In [15]:
# %env EB_AGENT_ACCESS_TOKEN=xxxxxxxx

import os

os.environ["EB_AGENT_ACCESS_TOKEN"] = "xxxxxxxx"

### 使用文心一言

首先，导入必要的依赖库。

In [16]:
import json
import asyncio
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

出于使用场景和性能的考虑，`ERNIE Bot Agent`只提供异步接口来调用文心一言模型。关于异步接口的详细介绍，请参考[asyncio文档](https://docs.python.org/3/library/asyncio.html)。

如下示例，我们首先创建文心一言`ernie-3.5`模型，然后两次调用`chat`接口传入只有单条`HumanMessage`的数组，文心一言模型会对单条`HumanMessage`做出回答，返回一条`AIMessage`。

In [17]:
model = ERNIEBot(model="ernie-3.5")  # 创建模型
human_message = HumanMessage(content="你好，你是谁")  # 定义输入信息
ai_message = await model.chat(messages=[human_message])  # 调用模型chat接口，非流式返回
print(ai_message.content, "\n")  # 输出结果

human_message = HumanMessage(content="推荐三个深圳有名的景点")  # 定义输入信息
ai_message = await model.chat(messages=[human_message], stream=True)  # 调用模型chat接口，流式返回
async for chunk in ai_message:  # 流式输出结果
    print(chunk.content, end="")

你好，我是文心一言，英文名是ERNIE Bot，可以协助你完成范围广泛的任务并提供有关各种主题的信息，比如回答问题，提供定义和解释及建议。如果你有任何问题，请随时向我提问。 

深圳是中国的一座经济特区，也是中国最现代化的城市之一，有许多值得游览的景点。以下是三个深圳有名的景点：

1. 深圳欢乐谷：深圳欢乐谷是一个主题公园，拥有各种现代化的游乐设施和表演节目，适合家庭游玩。这里有许多惊险刺激的过山车、碰碰车等项目，也有适合儿童玩的儿童区。此外，还有各种主题活动和节日庆典，如万圣节鬼屋、圣诞节狂欢等。
2. 深圳世界之窗：深圳世界之窗是一个微缩景观公园，展示了世界各地的著名建筑和景点，如埃菲尔铁塔、比萨斜塔、金字塔等。游客可以在这里欣赏到世界各地的文化和建筑风格，同时也可以参观各种主题展览和表演。
3. 深圳东部华侨城：深圳东部华侨城是一个大型旅游度假区，包括大峡谷、茶溪谷、云海谷等多个主题区。这里有各种刺激的游乐设施、主题酒店、购物中心等，是一个适合全家游玩的综合性度假胜地。

以上三个景点各有特色，为游客提供了不同的游玩体验，是深圳旅游的重要景点。

如果希望文心一言模型能够根据多轮对话的上下文进行回答，我们需要将前面对话的输入输出`Message`带入后面对话，具体参考如下代码。

In [18]:
model = ERNIEBot(model="ernie-3.5")
messages = []  # 使用列表保存所有Message信息

messages.append(HumanMessage(content="推荐三个深圳有名的景点"))
ai_message = await model.chat(messages=messages)
messages.append(ai_message)
print(ai_message.content, "\n")

messages.append(HumanMessage(content="根据你推荐的景点，帮我做一份一日游的攻略"))
ai_message = await model.chat(messages=messages)
messages.append(ai_message)
print(ai_message.content, "\n")

深圳是一个现代化的城市，有许多值得一游的景点。以下是三个深圳有名的景点推荐：

1. 深圳欢乐谷：这是一个大型的主题公园，拥有各种刺激和好玩的游乐设施，适合家庭游玩。此外，这里还有各种表演和活动，可以让人感受到浓郁的文化氛围。
2. 深圳东部华侨城：这是一个综合性的旅游度假区，拥有主题公园、温泉、高尔夫球场等设施，适合休闲和放松。这里的环境优美，让人可以远离城市的喧嚣，享受大自然的美妙。
3. 深圳海洋世界：这是一个展示海洋生物和海洋知识的主题公园，拥有各种海洋生物和水上活动，适合家庭和儿童游玩。这里不仅可以让人了解海洋知识，还可以让人感受到海洋的神奇和美妙。

这些景点都是深圳比较有名的旅游景点，各有特色，让人可以全面了解深圳的文化和历史，感受这个城市的魅力和活力。 

当然，以下是一份基于你推荐的景点的一日游攻略：

早上：

1. 早上8点左右出发，前往深圳欢乐谷。这里是中国主题公园的领军者，拥有大量的游乐设施，适合各个年龄段的人游玩。可以先游玩一些比较刺激的项目，如过山车、激流勇进等，然后看一场精彩的表演。
2. 中午：在欢乐谷内的餐厅用餐，品尝一些特色的美食。

下午：

1. 下午1点左右离开欢乐谷，前往深圳东部华侨城。这是一个综合性的旅游度假区，拥有优美的环境和各种休闲设施。可以先参观主题公园，了解一些海洋知识，然后去温泉放松一下身心。
2. 下午4点左右离开东部华侨城，前往深圳海洋世界。这是一个以海洋为主题的公园，可以近距离观察各种海洋生物，了解海洋的奥秘。

晚上：

1. 晚上6点左右离开海洋世界，去深圳的商业区逛逛，体验深圳的夜生活。这里有许多购物中心和美食街，可以购物和品尝美食。
2. 晚上8点左右返回酒店休息，结束一天的行程。

以上是一份参考的一日游攻略，具体的行程可以根据自己的兴趣和时间进行调整。 



下面示例，我们实现了一个简易的命令行聊天应用，可以和大语言模型网页端一样进行无限畅聊。

In [19]:
model = ERNIEBot(model="ernie-3.5")
messages = []

print("你好，有什么我可以帮助你的吗？ (输入q可以退出聊天)")
while True:
    prompt = input()
    if prompt == "q":
        break

    messages.append(HumanMessage(prompt))
    ai_message = await model.chat(messages=messages, stream=True)

    result = ""
    async for chunk in ai_message:
        result += chunk.content
        print(chunk.content, end="")
    print("")
    messages.append(AIMessage(result))

你好，有什么我可以帮助你的吗？ (输入q可以退出聊天)


 广东省的省会是什么


广东省的省会是**广州市**。广州市简称“穗”，别称羊城、花城，是广东省辖地级市、广东省省会、副省级市、国家中心城市、超大城市、广州都市圈核心城市，国务院批复确定的中国重要的中心城市、国际商贸中心和综合交通枢纽。


 这个城市简称什么


这个城市简称**穗**。


 q


下面示例，我们展示了文心一言模型如何使用FunctionCall。

In [20]:
# 定义function函数
def get_student_score(name: str) -> dict:
    info = {"小明": 80, "小红": 90, "小天": 95}
    if name in info:
        return {"score": info[name]}
    else:
        return f"we do not know the score of {name}"


# 定义function描述
functions = [
    {
        "name": "get_student_score",
        "description": "查询学生的分数",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "学生姓名",
                },
            },
            "required": [
                "name",
            ],
        },
        "responses": {
            "type": "object",
            "properties": {
                "score": {
                    "type": "integer",
                    "description": "分数",
                },
            },
        },
    },
]


model = ERNIEBot(model="ernie-3.5")
messages = []

messages.append(HumanMessage("请问小明的分数是多少"))
ai_message = await model.chat(messages=messages, functions=functions)  # 发送问题，带上function描述
messages.append(ai_message)

function_call = ai_message.function_call
if function_call is not None:  # 如果返回的AIMessage有触发function，会有function_call字段
    name = function_call["name"]  # function_call的函数名称
    arguments = eval(function_call["arguments"])  # function_call的函数输入实参
    result = eval(name)(**arguments)  # 使用函数实参，调用并执行函数，拿到结果
    print("function_call:", function_call)
    print("function result:", result)

    function_message = FunctionMessage(
        name=name, content=json.dumps(result, ensure_ascii=False)
    )  # 构建FunctionMessage，封装函数的结果
    messages.append(function_message)

    ai_message = await model.chat(messages=messages, functions=functions)  # 将函数结果返回给模型，进行润色，得到最终输出
    print("final result:", ai_message.content)

function_call: {'name': 'get_student_score', 'thoughts': '用户想要查询小明的分数，我需要使用查询学生分数的工具。', 'arguments': '{"name":"小明"}'}
function result: {'score': 80}
final result: 根据我的查询，小明的分数是80分。如果你有其他关于小明的成绩或者其他方面的问题，请随时告诉我。
